In [1]:
import pandas as pd 
import numpy as np
import geopandas as gpd
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from shapely.geometry import Point
from shapely.geometry import Polygon

In [2]:
# Routine to verify if the BGRI code is the original polygon
def verifyBGRIconsistency():
    notInPolygon = []
    bgriInOriginal = gdf_escolas_1CEB_BGRI['BGRI11'].value_counts().keys().tolist()
    for e in bgriInOriginal:
        if e not in gdf_BGRI_2011.index:
            notInPolygon.append(e)
    return notInPolygon

### Leitura de Ficheiros

In [3]:
gdf_BGRI_2011 = gpd.read_file('./data/BGRI_2011/CONTINENTE/BGRI11_CONT.shp')
gdf_BGRI_2011['Code_Freg'] = gdf_BGRI_2011['DTMN11'] + gdf_BGRI_2011['FR11']


In [4]:
df_correspond_nuts3 = pd.read_csv("./data/00_Base_de_dados/CODIGOS_MUNICIPIOS_NUTS3.csv",
        engine='python', delimiter= ",", dtype= {'code_mun': str})
gdf_BGRI_2011 = gdf_BGRI_2011.merge(df_correspond_nuts3[['code_mun','nome_mun']], right_on= 'code_mun', left_on = 'DTMN11', how = 'left')
gdf_BGRI_2011.drop(columns = ['code_mun'], inplace = True)

In [5]:
gdf_escolas_Portugal_1CEB = gpd.read_file('./data/00_Base_de_dados/escolas_portugal/gdf_escolas_publicas_1CEB.shp')
gdf_escolas_Portugal_1CEB = gdf_escolas_Portugal_1CEB.to_crs({'init': 'epsg:3763'})

In [6]:
gdf_escolas_Portugal_1CEB.ESCOLA.count()

3426

In [7]:
df_TIPAU = pd.read_excel("./data/00_Base_de_dados/TIPAU_Freguesia_2011.xlsx",sheet_name='SIG')
gdf_BGRI_2011 = gdf_BGRI_2011.merge(df_TIPAU[['Código','TIPAU']],
left_on='Code_Freg', right_on='Código')

In [8]:
#gdf_BGRI_2011[gdf_BGRI_2011.LUG11=='999999']

In [9]:
df_BGRI_2011_Grupo_etario = pd.read_csv("./data/00_Base_de_dados/C2011_subsec_Grupo_Etario.csv", delimiter= ',', 
dtype ={'MUNICIPIO': str, 'FREGUESIA': str, 'SECCAO': str, 'SUBSECCAO': str})

/opt/conda/envs/pandana/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
df_BGRI_Dados_gerais = pd.read_csv("./data/00_Base_de_dados/BGRI2011_PT.csv", 
sep = ';', dtype= {'GEO_COD': str},  engine = 'python')
df_BGRI_Dados_gerais['GEO_COD'] = df_BGRI_Dados_gerais['GEO_COD'].str[1:]

In [11]:
df_Accessibility_BGRI = pd.read_excel('./data/00_Base_de_dados/finalResultsAccessbility60km.xlsx', 
                                      dtype= {'Code_Freg': str, 'key_0': str})
df_Accessibility_BGRI['code_mun']=df_Accessibility_BGRI['key_0'].astype(str).str[0:4]

Calcular pop desempregada

In [12]:
df_BGRI_Dados_gerais['Pop_desempregada']= df_BGRI_Dados_gerais.loc[:,['N_IND_RESID_DESEMP_PROC_1EMPRG','N_IND_RESID_DESEMP_PROC_EMPRG']].sum(axis=1)
df_BGRI_Desempregados = df_BGRI_Dados_gerais.loc[:, ['GEO_COD', 'Pop_desempregada']]

In [13]:
df_BGRI_2011_Grupo_etario ['Pop_idade_ativa'] = df_BGRI_2011_Grupo_etario.loc[:,'15 - 19':'60 - 64'].sum(axis = 1)

In [14]:
gdf_BGRI_2011 = gdf_BGRI_2011.merge(df_BGRI_2011_Grupo_etario\
[['SUBSECCAO','Total Of CountOfIND', 'Pop_idade_ativa']][df_BGRI_2011_Grupo_etario.SEXO==0],
left_on='BGRI11', right_on='SUBSECCAO', how='left')

#what.fillna('unknown', inplace=True)

In [15]:
gdf_BGRI_2011[gdf_BGRI_2011.LUG11=='999999'].shape

(19009, 17)

In [16]:
gdf_BGRI_2011.rename(columns={"Total Of CountOfIND": "Pop_Total_2011"}, inplace =True) 

In [17]:
gdf_BGRI_2011 = gdf_BGRI_2011.merge(df_BGRI_Desempregados,
left_on='BGRI11', right_on='GEO_COD', how='left')

In [18]:
gdf_BGRI_2011['Tx_desemprego'] = gdf_BGRI_2011['Pop_desempregada']/gdf_BGRI_2011['Pop_idade_ativa']*100

In [19]:
gdf_escolas_Portugal_1CEB.head()

,DSR,DISTRITO,CONCELHO,NUTIII,CÓD. AGRUP,AGRUPAMENT,CÓD. DGEEC,CÓD. DGPGF,ESCOLA,MORADA,...,CODCONCELH,CODDISTRIT,CODNUTIII,CODGEPESED,CODDRE,ENSINOS_LI,LONG,Unnamed_ 3,CODCONCE_1,geometry
0,Norte,Viana do Castelo,Melgaço,Minho Lima,152602.0,Escolas de Melgaço,1603773,294329,Escola Básica de Melgaço,Avenida Capitão Salgueiro Maia,...,1603,16.0,10101.0,1603190.0,101.0,AB,-8.260877,Público,None,POINT (-10567.991 270827.129)
1,Norte,Viana do Castelo,Monção,Minho Lima,153023.0,Escolas de Monção,1604149,291663,"Escola Básica José Pinheiro Gonçalves, Monção",Bairro das Escolas Primárias,...,1604,16.0,10101.0,1604079.0,101.0,AB,-8.476451,Público,None,POINT (-28412.705 267350.864)
2,Norte,Viana do Castelo,Melgaço,Minho Lima,152602.0,Escolas de Melgaço,1603780,265342,"Escola Básica de Pomares, Melgaço",Pomares,...,1603,16.0,10101.0,1603190.0,101.0,AB,-8.277731,Público,None,POINT (-11970.880 265590.835)
3,Norte,Viana do Castelo,Monção,Minho Lima,153023.0,Escolas de Monção,1604542,291614,"Escola Básica de Estrada, Monção",Estrada,...,1604,16.0,10101.0,1604079.0,101.0,AB,-8.488950,Público,None,POINT (-29457.051 264948.659)
4,Norte,Viana do Castelo,Valença,Minho Lima,150587.0,"Escolas de Muralhas do Minho, Valença",1608974,294330,"Escola Básica de Friestas, Valença",Rua Amaro da Costa - Trofa de Cima,...,1608,16.0,10101.0,1608480.0,101.0,AB,-8.571300,Público,None,POINT (-36279.268 263972.853)


Fazer spatial join entres escolas e BGRI

In [20]:
gdf_escolas_1CEB_BGRI = gpd.sjoin(gdf_escolas_Portugal_1CEB, gdf_BGRI_2011[['BGRI11','geometry']], op= 'within', how='left')

In [21]:
# ----------------- Verification CODE ------------------------ #
gdf_BGRI_2011.index = gdf_BGRI_2011.BGRI11
checkBGRI = verifyBGRIconsistency()
len(checkBGRI) 

0

In [22]:
gdf_escolas_1CEB_BGRI.shape

(3426, 34)

In [23]:
gdf_escolas_Portugal_1CEB.ESCOLA.count()

3426

In [24]:
gdf_escolas_Portugal_1CEB.crs

{'init': 'epsg:3763'}

In [25]:
gdf_BGRI_2011.crs

{'init': 'epsg:3763'}

Este join está a dar pau...

In [26]:
# gdf_BGRI_2011.index = gdf_BGRI_2011.SUBSECCAO <=========== CULPADO FILHO DA PUTA
gdf_BGRI_2011['N_escolas'] = gdf_escolas_1CEB_BGRI['BGRI11'].value_counts()

In [27]:
# ----------------- Verification CODE ------------------------ #
gdf_BGRI_2011.index = gdf_BGRI_2011.BGRI11
checkBGRI = verifyBGRIconsistency()
len(checkBGRI) 

0

In [28]:
gdf_escolas_1CEB_BGRI['BGRI11'].value_counts().sum()

3426

In [29]:
gdf_BGRI_2011[gdf_BGRI_2011.N_escolas.notna()].N_escolas.sum()

3426.0

Adicionar media, max, etc. em todos os campos e para diferetns unidades (freguesia, municipio)

In [30]:
df_Accessibility_BGRI_2 = df_Accessibility_BGRI.copy()
methods = ['mean', 'max', 'min', 'std']
for i in methods:
    df_Accessibility_BGRI_2 ['Dist_freguesia_' + i] = df_Accessibility_BGRI_2.groupby('Code_Freg')['Dist_1'].\
    transform(i)

for i in methods:
    df_Accessibility_BGRI_2 ['Dist_municipio_' + i] = df_Accessibility_BGRI_2.groupby('code_mun')['Dist_1'].\
    transform(i)

df_Accessibility_BGRI_2
#df_alunos_subseccao.groupby('Code_Freg')[column_names].transform('sum'))

,id,key_0,Code_Freg,Nome_Freg,Dist_1,Dist_2,Dist_3,IQ,Diference_1_2,Diference_2_3,code_mun,Dist_freguesia_mean,Dist_freguesia_max,Dist_freguesia_min,Dist_freguesia_std,Dist_municipio_mean,Dist_municipio_max,Dist_municipio_min,Dist_municipio_std
0,152643,15110100101,151101,Sesimbra (Castelo),4160.160245,4160.160245,4160.160245,4.480000,0.000000,0.000000,1511,2057.577067,6762.897225,57.959999,1373.860782,1828.027084,6762.897225,57.959999,1376.635944
1,101852,15070500412,150705,Alto-Estanqueiro-Jardia,754.928680,754.928680,754.928680,6.413043,0.000000,0.000000,1507,931.820451,2046.949036,125.047666,461.432369,777.320428,7623.784424,13.059000,666.705818
2,101772,15070500104,150705,Alto-Estanqueiro-Jardia,827.850708,854.534563,854.534563,6.863014,26.683855,0.000000,1507,931.820451,2046.949036,125.047666,461.432369,777.320428,7623.784424,13.059000,666.705818
3,101816,15070500303,150705,Alto-Estanqueiro-Jardia,1191.199666,1227.836019,1227.836019,4.723404,36.636353,0.000000,1507,931.820451,2046.949036,125.047666,461.432369,777.320428,7623.784424,13.059000,666.705818
4,183810,03120100112,031201,Abade de Vermoim,2364.897949,2417.013916,2417.013916,5.875000,52.115967,0.000000,0312,1994.285339,2478.447998,1483.285034,358.814155,1031.359615,3594.336442,0.000000,556.178584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201079,126427,17080400601,170804,Godim,540.600386,848.677002,60000.000000,10.313725,308.076616,59151.322998,1708,991.819543,3214.917969,103.960168,802.630890,7119.664968,17826.741089,103.960168,6137.644293
201080,126465,17080700303,170807,Peso da Régua,348.976500,792.191635,60000.000000,8.791667,443.215136,59207.808365,1708,1099.503093,2852.753784,245.679500,644.810789,7119.664968,17826.741089,103.960168,6137.644293
201081,126426,17080400505,170804,Godim,554.218239,774.685257,60000.000000,7.400000,220.467018,59225.314743,1708,991.819543,3214.917969,103.960168,802.630890,7119.664968,17826.741089,103.960168,6137.644293
201082,126424,17080400503,170804,Godim,426.961401,696.877002,60000.000000,8.660870,269.915601,59303.122998,1708,991.819543,3214.917969,103.960168,802.630890,7119.664968,17826.741089,103.960168,6137.644293


In [31]:
# df_Accessibility_BGRI_2.to_csv('./data/accessibilityAndSocialMetrics.csv')

In [32]:
df_Accessibility_BGRI_2 = df_Accessibility_BGRI_2.rename(columns={'key_0':'BGRI'})
df_Accessibility_BGRI_2.index = df_Accessibility_BGRI_2.BGRI
df_Accessibility_BGRI_2 = df_Accessibility_BGRI_2.drop(columns=['BGRI'])

In [33]:
df_Accessibility_BGRI_2.columns

Index(['id', 'Code_Freg', 'Nome_Freg', 'Dist_1', 'Dist_2', 'Dist_3', 'IQ',
       'Diference_1_2', 'Diference_2_3', 'code_mun', 'Dist_freguesia_mean',
       'Dist_freguesia_max', 'Dist_freguesia_min', 'Dist_freguesia_std',
       'Dist_municipio_mean', 'Dist_municipio_max', 'Dist_municipio_min',
       'Dist_municipio_std'],
      dtype='object')

In [34]:
gdf_BGRI_2011.columns

Index(['OBJECTID', 'DTMN11', 'FR11', 'SEC11', 'SS11', 'BGRI11', 'LUG11',
       'LUG11DESIG', 'area', 'geometry', 'Code_Freg', 'nome_mun', 'Código',
       'TIPAU', 'SUBSECCAO', 'Pop_Total_2011', 'Pop_idade_ativa', 'GEO_COD',
       'Pop_desempregada', 'Tx_desemprego', 'N_escolas'],
      dtype='object')

In [54]:
df_Accessibility_BGRI_Final = pd.merge(df_Accessibility_BGRI_2, gdf_BGRI_2011[['nome_mun','LUG11','DTMN11','Code_Freg','Pop_Total_2011','Pop_idade_ativa','Pop_desempregada', 'Tx_desemprego', 'N_escolas', 'TIPAU']], left_on=[df_Accessibility_BGRI_2.index], right_on=[gdf_BGRI_2011.index], how='outer')

In [55]:
df_Accessibility_BGRI_Final.N_escolas.sum()

3426.0

In [56]:
df_Accessibility_BGRI_Final[df_Accessibility_BGRI_Final['Dist_1'].isna()]

,key_0,id,Code_Freg_x,Nome_Freg,Dist_1,Dist_2,Dist_3,IQ,Diference_1_2,Diference_2_3,...,nome_mun,LUG11,DTMN11,Code_Freg_y,Pop_Total_2011,Pop_idade_ativa,Pop_desempregada,Tx_desemprego,N_escolas,TIPAU
201084,04070200103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Mirandela,013810,0407,040702,9.0,2.0,1,50.000000,NaN,APR
201085,04070200108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Mirandela,999999,0407,040702,NaN,NaN,0,NaN,NaN,APR
201086,04072900103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Mirandela,013866,0407,040729,37.0,24.0,1,4.166667,NaN,APR
201087,04072900104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Mirandela,013866,0407,040729,28.0,17.0,0,0.000000,NaN,APR
201088,04070300103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Mirandela,034316,0407,040703,4.0,1.0,0,0.000000,NaN,APR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255953,01010100112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,�gueda,014422,0101,010101,11.0,9.0,0,0.000000,NaN,APR
255954,01010100107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,�gueda,014427,0101,010101,6.0,6.0,0,0.000000,NaN,APR
255955,01010100102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,�gueda,014420,0101,010101,19.0,17.0,0,0.000000,NaN,APR
255956,01010100101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,�gueda,014420,0101,010101,9.0,4.0,0,0.000000,NaN,APR


In [57]:
df_Accessibility_BGRI_Final = df_Accessibility_BGRI_Final.drop(columns=['Dist_2', 'Dist_3', 'Diference_1_2', 'Diference_2_3'])


In [59]:
# é que já tem o Nome_Freg, mas vai buscar só os valores para as BGRI que estão na acessibilidade
# e falta também acrescentar o ['mean', 'max', 'min', 'std'], por freguesia e por município em todas as BGRI

In [60]:
df_Accessibility_BGRI_Final.to_csv('./data/accessibilityAndSocialMetrics.csv')

### Later Manipulation to get missing BGRI's

In [32]:
# Load Previous Results
result = pd.read_csv('./data/acess_results/accessibilityAndSocialMetrics_withCode_Freg.csv', dtype={'key_0':str})
result.index = result.key_0
result = result.drop(columns=['key_0','Unnamed: 0'])

/home/fillipe/anaconda3/envs/pandana/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
# Select only the values with NaN values in Dist_1
missingAccess = result[result['Dist_1'].isna()]

In [34]:
# Get polygons from Original 2011 BGRI data
missingAccess = pd.merge(missingAccess, gdf_BGRI_2011[['BGRI11','geometry']], how='left', left_on=[missingAccess.index], right_on=[gdf_BGRI_2011.BGRI11])
# Transform missinAcces in GPD
missingAccess = gpd.GeoDataFrame(missingAccess)

In [40]:
# Adjust DF index
missingAccess.index = missingAccess.BGRI11

### Start Pandana Logic

In [43]:
result.loc['15100506730']

,id,Code_Freg_x,Nome_Freg,Dist_1,IQ,code_mun,Dist_freguesia_mean,Dist_freguesia_max,Dist_freguesia_min,Dist_freguesia_std,...,nome_mun,LUG11,DTMN11,Code_Freg_y,Pop_Total_2011,Pop_idade_ativa,Pop_desempregada,Tx_desemprego,N_escolas,TIPAU
key_0,,,,,,,,,,,,,,,,,,,,,
15100506730,150313.0,151005.0,Corroios,3584.033147,0.0,1510.0,1617.275835,5000.2771,19.823,1451.090311,...,Seixal,999999,1510,151005,NaN,NaN,0,NaN,NaN,APU
15100506730,10806.0,151005.0,Corroios,1731.716992,0.0,1510.0,1617.275835,5000.2771,19.823,1451.090311,...,Seixal,999999,1510,151005,NaN,NaN,0,NaN,NaN,APU


In [53]:
result_old = pd.read_csv('./data/finalResultsAccessbility60km.csv', dtype={'BGRI':str})

In [54]:
result_old[result_old['key_0']=='15100506730']

,Unnamed: 0,2,BGRI,IQ,Unnamed: 0.1,accessN1,accessN3,key_0,1,2.1,3


In [55]:
result_old

,Unnamed: 0,2,BGRI,IQ,Unnamed: 0.1,accessN1,accessN3,key_0,1,2.1,3
0,0,12840.260010,14010100103,2.444444,0.0,8804.343018,13261.742676,NaN,NaN,NaN,NaN
1,1,12355.741699,14010100104,3.333333,1.0,8319.824707,12777.225098,NaN,NaN,NaN,NaN
2,2,12224.947266,14010100105,4.000000,2.0,8189.029785,12646.429688,NaN,NaN,NaN,NaN
3,3,11065.426107,14010100106,6.444444,3.0,6393.092285,11486.908854,NaN,NaN,NaN,NaN
4,4,10889.387207,14010100107,5.857143,4.0,6853.469971,11310.869629,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
185269,185269,NaN,NaN,6.521739,NaN,NaN,NaN,1.401190e+10,462.447998,11780.381836,11859.259766
185270,185270,NaN,NaN,6.800000,NaN,NaN,NaN,1.401190e+10,525.518501,11843.452393,11922.330566
185271,185271,NaN,NaN,6.714286,NaN,NaN,NaN,1.401190e+10,1451.594971,12969.017578,13047.896484
185272,185272,NaN,NaN,7.500000,NaN,NaN,NaN,1.401190e+10,1394.891589,12499.522070,12578.400195


In [57]:
gdf_BGRI_2011[gdf_BGRI_2011['BGRI11']=='15100506730']

,OBJECTID,DTMN11,FR11,SEC11,SS11,BGRI11,LUG11,LUG11DESIG,area,geometry,Code_Freg
206786,198429,1510,05,067,30,15100506730,999999,Residual,0.586905,POINT (-89809.898 -118812.357),151005


In [59]:
result[result.index.duplicated()]

,id,Code_Freg_x,Nome_Freg,Dist_1,IQ,code_mun,Dist_freguesia_mean,Dist_freguesia_max,Dist_freguesia_min,Dist_freguesia_std,...,nome_mun,LUG11,DTMN11,Code_Freg_y,Pop_Total_2011,Pop_idade_ativa,Pop_desempregada,Tx_desemprego,N_escolas,TIPAU
key_0,,,,,,,,,,,,,,,,,,,,,
15110302932,150984.0,151103.0,Quinta do Conde,1912.229980,0.000000,1511.0,2330.572189,5725.398600,1432.456055,1228.695743,...,Sesimbra,999999,1511,151103,NaN,NaN,0,NaN,NaN,APU
09111800119,140984.0,91118.0,Malcata,10062.630967,0.000000,911.0,8897.886802,21632.111328,7310.143555,3133.152266,...,Sabugal,999999,911,91118,NaN,NaN,0,NaN,NaN,APR
05070700118,140568.0,50707.0,Meimão,11025.513672,3.833333,507.0,18935.794259,21398.463728,11025.513672,2141.031495,...,Penamacor,999999,507,50707,8.0,3.0,0,0.000000,NaN,APR
05071200115,140569.0,50712.0,Vale da Senhora da Póvoa,11025.417969,0.000000,507.0,14845.776480,15804.986654,11025.417969,1087.940127,...,Penamacor,999999,507,50712,NaN,NaN,0,NaN,NaN,APR
05070600120,140567.0,50706.0,Benquerença,15813.665039,3.666667,507.0,10407.901594,15813.665039,9188.749023,1541.218123,...,Penamacor,999999,507,50706,12.0,5.0,0,0.000000,NaN,APR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
02080200420,58697.0,20802.0,Ferreira do Alentejo,3169.264244,9.250000,208.0,2458.290936,12976.755859,37.098333,3229.624980,...,Ferreira do Alentejo,999999,208,20802,20.0,16.0,2,12.500000,NaN,AMU
10150100733,115624.0,101501.0,Abiul,8031.436686,2.666667,1015.0,3969.102759,9403.430664,31.150001,1910.984280,...,Pombal,999999,1015,101501,3.0,1.0,0,0.000000,NaN,APR
10050500142,28428.0,100505.0,Pó,1219.520063,8.153846,1005.0,655.641827,3841.452881,179.806330,669.382503,...,Bombarral,999999,1005,100505,21.0,13.0,1,7.692308,NaN,AMU


In [60]:
result.loc['14130300246']

,id,Code_Freg_x,Nome_Freg,Dist_1,IQ,code_mun,Dist_freguesia_mean,Dist_freguesia_max,Dist_freguesia_min,Dist_freguesia_std,...,nome_mun,LUG11,DTMN11,Code_Freg_y,Pop_Total_2011,Pop_idade_ativa,Pop_desempregada,Tx_desemprego,N_escolas,TIPAU
key_0,,,,,,,,,,,,,,,,,,,,,
14130300246,93459.0,141303.0,Cardigos,13854.742258,5.47619,1413.0,14125.969708,19113.634394,5692.805176,3125.765963,...,Ma��o,999999,1413,141303,21.0,2.0,0,0.0,NaN,APR
14130300246,136784.0,141303.0,Cardigos,5692.805176,5.47619,1413.0,14125.969708,19113.634394,5692.805176,3125.765963,...,Ma��o,999999,1413,141303,21.0,2.0,0,0.0,NaN,APR


In [62]:
gdf_BGRI_2011[gdf_BGRI_2011['BGRI11']=='14130300246']

,OBJECTID,DTMN11,FR11,SEC11,SS11,BGRI11,LUG11,LUG11DESIG,area,geometry,Code_Freg
9830,9831,1413,03,002,46,14130300246,999999,Residual,21.1921,POINT (13514.195 4331.110),141303


In [66]:
all_last = pd.read_excel('data/accessibilityAndSocialMetrics_ALL_last.xlsx', dtype={'key_0':str, 'DTMN11':str})


In [70]:
all_last[all_last['key_0']=='14130300246']

,nID,key_0,id,LUG11,DTMN11,Pop_Total_2011,Pop_idade_ativa,Pop_desempregada,Tx_desemprego,IQ,...,Dist_freguesia_std,IQ_freguesia,code_mun,nome_mun,Dist_municipio_mean,Dist_municipio_max,Dist_municipio_min,Dist_municipio_std,IQ_municipio,N_escolas
93459,192534.0,14130300246,93459.0,NaN,1413,21.0,2.0,0.0,0.0,5.47619,...,3125.765963,4.611173,1413.0,Mação,8743.325918,19113.634394,146.186664,4717.167109,4.132143,NaN
136784,192535.0,14130300246,136784.0,NaN,1413,21.0,2.0,0.0,0.0,5.47619,...,3125.765963,4.611173,1413.0,Mação,8743.325918,19113.634394,146.186664,4717.167109,4.132143,NaN
